In [26]:
# Dependencies
from bs4 import BeautifulSoup as bs
import pandas as pd
from splinter import Browser
import requests
import time
import pymongo
# from selenium import webdriver
# from selenium.webdriver.common.keys import Keys

In [29]:
nasa_mars_news="https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

response = requests.get(nasa_mars_news)

# Create BeautifulSoup object; parse with 'html.parser'
soup1 = bs(response.text, 'html.parser')

news_title = soup1.find(class_="content_title").a.text.replace("\'", " ").replace("'", ' ').strip()
news_title

news_p = soup1.find(class_="rollover_description_inner").text.replace("\'", " ").replace("'", ' ').strip()
news_p

'NASA chose a seventh-grader from Virginia as winner of the agency s "Name the Rover" essay contest. Alexander Mather s entry for "Perseverance" was voted tops among 28,000 entries.'

In [30]:
# this tells python to control chrome
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [32]:
html = browser.html
# Parse HTML with Beautiful Soup
soup2 = bs(html, 'html.parser')

browser.links.find_by_partial_text('FULL').click()

In [33]:
time.sleep(4)

browser.links.find_by_partial_href('spaceimages/details').click()

In [34]:
new_html = browser.html
soup3 = bs(new_html, "html.parser")
# print(soup3.prettify())

In [35]:
img_tag = soup3.find('figure', class_='lede').a
    
featured_image_url = f"https://www.jpl.nasa.gov{img_tag['href']}"
# print(featured_image_url)

In [36]:
# Comments in this block and next from a selenium solution for filtering weather-only tweets and selecting one:
# I was told my selector was invalid or illegal, I'm guessing the second one. Might have worked otherwise.
# Source:
# (https://medium.com/analytics-vidhya/create-your-own-twitter-dataset-with-this-simple-python-scraper-710bf7c5dc04)

# browser=webdriver.Chrome()

twitter_url="https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)

In [41]:
time.sleep(4)

html = browser.html
soup4 = bs(html, 'html.parser')
# retrieval of generic tag
mars_weather=soup4.find("article").text.replace('\n', ' ').replace(',', ' ').strip()

# mars_weather=[]
# for result in browser.find_elements_by_class_name("^css-1dbjc4n"):
#     try:
#         text = browser.find_by_partial_text("InSight sol")
        
#         if text:
#             mars_weather.append(text)
            
#         else: print("if nope!")
            
#     except: print("try nope!")

mars_weather

"Mars Weather RetweetedTony Rice@rtphokie·14hWhile the Smithsonian is closed  we can peruse the National @airandspace Museum's online collection   Here's one of my favorites: https://airandspace.si.edu/collections/14"

In [11]:
facts_url="https://space-facts.com/mars/"
mars_facts = pd.read_html(facts_url)
mars_facts = mars_facts[0]
mars_facts = mars_facts.rename(columns={0: "Description", 1: "Value"})
mars_facts

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
mars_facts_dict = mars_facts.set_index('Description')['Value'].to_dict()
mars_facts_dict

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

In [13]:
hemispheres_url='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [14]:
new_html=browser.html
soup5 = bs(new_html, "html.parser")
# print(soup5.prettify())

In [15]:
# Retrieve the parent divs containing all hemisphere photos
results = soup5.find_all('div', class_='item')

headers = []
for result in results:
    # scrape the hemisphere photo titles
    title = result.find('h3').text
    headers.append(title)
print(headers)
    
hemispheres = []
for header in headers:
    hemi_name = header.split(' ')[0]
    hemispheres.append(hemi_name)
print(hemispheres)
    
def first_lower(s):
   if len(s) == 0:
      return s
   else:
      return s[0].lower() + s[1:]

url_hemi_names = []
for hemisphere in hemispheres:
    lowercased = first_lower(hemisphere)
    url_hemi_names.append(lowercased)
print(url_hemi_names)

img_urls=[]
for name in url_hemi_names:
    browser.visit(hemispheres_url)
    current = browser.links.find_by_partial_href(f'Viking/{name}')['href']
    soup = browser.visit(current)

    url = browser.find_by_text('Sample')['href']
    img_urls.append(url)
    
print(img_urls)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']
['Cerberus', 'Schiaparelli', 'Syrtis', 'Valles']
['cerberus', 'schiaparelli', 'syrtis', 'valles']
['http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg', 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg']


In [16]:
hemisphere_image_urls = [
    {"title": headers[0], "img_url": img_urls[0]},
    {"title": headers[1], "img_url": img_urls[1]},
    {"title": headers[2], "img_url": img_urls[2]},
    {"title": headers[3], "img_url": img_urls[3]},   
]
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [17]:
master_dict = {
    "collection_timestamp":time.time(),
    "news_title":news_title,
    "news_p":news_p,
    "featured_image_url":featured_image_url,
    "mars_weather":mars_weather,
    "mars_facts":mars_facts_dict,
    "hemisphere_info":hemisphere_image_urls
}
master_dict

{'news_title': "Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover",
 'news_p': 'NASA chose a seventh-grader from Virginia as winner of the agency\'s "Name the Rover" essay contest. Alexander Mather\'s entry for "Perseverance" was voted tops among 28,000 entries.',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18840_hires.jpg',
 'mars_weather': 'Mars Weather@MarsWxReport·Mar 10InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)\nwinds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)\npressure at 6.30 hPa1624',
 'mars_facts': {'Equatorial Diameter:': '6,792 km',
  'Polar Diameter:': '6,752 km',
  'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
  'Moons:': '2 (Phobos & Deimos)',
  'Orbit Distance:': '227,943,824 km (1.38 AU)',
  'Orbit Period:': '687 days (1.9 years)',
  'Surface Temperature:': '-87 to -5 °C',
  'First Record:': '2nd millennium BC',
  'Recorded By:': 'Egyptian astronomers'},
 'hem

In [18]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [19]:
# Define database and collection
db = client.mars_db
collection = db.mars_master_info

In [20]:
# Inserting currently scraped data into the mars_master_info collection
collection.insert_one(master_dict)

In [36]:
# Display items in MongoDB collection
current_mars = db.mars_master_info.find()

for i in current_mars:
        print(i)
        
# and it printed the contents of the master dictionary! yay!

{'_id': ObjectId('5e6c1300ec66cf891554016b'), 'collection_timestamp': 1584141056.2174249, 'news_title': "Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover", 'news_p': 'NASA chose a seventh-grader from Virginia as winner of the agency\'s "Name the Rover" essay contest. Alexander Mather\'s entry for "Perseverance" was voted tops among 28,000 entries.', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18249_hires.jpg', 'mars_weather': 'Mars Weather RetweetedNASA InSight@NASAInSight·6hA bit of good news from #Mars: our new approach of using the robotic arm to push the mole appears to be working! The teams @NASAJPL/@DLR_en are excited to see the images and plan to continue this approach over the next few weeks.  #SaveTheMole  \n\nFAQ: http://go.nasa.gov/HP3FAQ0:10206.5K views1116544.4K', 'mars_facts': {'Equatorial Diameter:': '6,792 km', 'Polar Diameter:': '6,752 km', 'Mass:': '6.39 × 10^23 kg (0.11 Earths)', 'Moons:': '2 (Phobos & De